<a href="https://colab.research.google.com/github/marco-sorich/SeasonalityChartAnalysis/blob/main/Seasonality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Seasonalität

_Notebook zur Analyse von Seasonalität eines Symbols._


### Anforderungen

<sub><sup>umgesetzt → &#x2705; / in Arbeit → &#x23F3;</sup></sub>

1. Implementierung über statsmodels: https://www.statsmodels.org/dev/examples/notebooks/generated/statespace_seasonal.html
1. Tägliche Linien-Grafik
1. Zusätzlich monatliche/wochentägliche Balken-Grafik
1. Seasonalität per Machine Learning: https://machinelearningmastery.com/time-series-seasonality-with-python/
1. Akkummulierte Werte bei Kauf/Verkauf
1. "Heute"-Linie einblenden
1. Mit/ohne Indexierung auf 100
1. Beschränkung der Zeitspanne auf max 30 Jahre
1. Einstellbare Beschränkung der Zeitspanne
1. Darstellung Hüllkurve oder Streuung
1. Fernere Vergangenheit anders darstellen als nähere
1. Interaktive Grafik (Fadenkreuz bei Maus)??

In [2]:
! pip install yfinance
! pip install pandas
! pip install pandas_datareader

import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

ModuleNotFoundError: No module named 'pandas'

### Symbol auswählen und einlesen


In [ ]:
#symbol='2B7K.DE'   # iShares MSCI World SRI UCITS ETF EUR (Acc)
symbol='EUNL.DE'    # iShares Core MSCI World UCITS ETF USD (Acc)

yf.pdr_override() # <== that's all it takes :-)
from pandas_datareader import data as pdr
df = pdr.get_data_yahoo(tickers=[symbol], interval="1d")
df.head()

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- EUNL.DE: No timezone found, symbol may be delisted


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


### Ausgabe Schlusskurse und Gleitender Durchschnitt

In [ ]:
rolling_resolution = 100

plt.figure(figsize=(20,8))
plt.xlabel('Data', fontsize=17)
plt.ylabel('Price', fontsize=17)
plt.title('Daily closing price', fontsize=17)
#plt.grid(b=True, which='major', color='b', linestyle='-')

df['Close'].plot(legend=True, label='close price')
df['Close'].rolling(rolling_resolution).mean().plot(legend=True, label=str(rolling_resolution) + '-day moving average')
#df[symbol].plot(legend=True, label='20-day VWAP')

plt.show()

NameError: name 'plt' is not defined